In [2]:
!pip install pandas --trusted-host pypi.org --trusted-host files.pythonhosted.org
!pip install openpyxl --trusted-host pypi.org --trusted-host files.pythonhosted.org
!pip install rapidfuzz --trusted-host pypi.org --trusted-host files.pythonhosted.org
!pip install feedparser --trusted-host pypi.org --trusted-host files.pythonhosted.org
import sys
!{sys.executable} -m pip install rapidfuzz --trusted-host pypi.org --trusted-host files.pythonhosted.org
!{sys.executable} -m pip install openpyxl --trusted-host pypi.org --trusted-host files.pythonhosted.org
!{sys.executable} -m pip install networkx --trusted-host pypi.org --trusted-host files.pythonhosted.org
import pandas as pd
from rapidfuzz import fuzz
import networkx as nx
import re
from urllib.parse import urlparse, urlunparse
import requests
from bs4 import BeautifulSoup
import time
import random
from urllib.parse import urljoin
pd.set_option("display.max_colwidth", None)
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import concurrent.futures
from tqdm import tqdm

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 24.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 24.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 24.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 24.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 24.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 24.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Cleaning Brand names

In [28]:
INPUT_FILE = "brands_for_cleaning.xlsx"
OUTPUT_FILE = "brands_duplicates_clusters.xlsx"
SIMILARITY_THRESHOLD = 88

df = pd.read_excel(INPUT_FILE)
df["Бренд"] = df["Бренд"].astype(str)

def clean_name(name):
    name = str(name).lower()
    for ch in ["'", "’", "-", ",", ".", '"']:
        name = name.replace(ch, "")
    name = " ".join(name.split())
    return name

df["Название_очищенное"] = df["Бренд"].apply(clean_name)

names = df["Название_очищенное"].tolist()
original_names = df["Бренд"].tolist()

C:\Users\varva\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Making clusters to reveal the Brand names duplicates

In [29]:
# ===== граф =====
G = nx.Graph()

for i in range(len(names)):
    G.add_node(original_names[i])
    for j in range(i + 1, len(names)):
        score = fuzz.token_sort_ratio(names[i], names[j])
        if score >= SIMILARITY_THRESHOLD:
            G.add_edge(original_names[i], original_names[j])

# ===== кластеры =====
clusters = list(nx.connected_components(G))

cluster_list = []
for idx, cluster in enumerate(clusters, start=1):
    cluster_list.append({
        "Кластер": idx,
        "Кол-во брендов": len(cluster),
        "Бренды": ", ".join(str(b) for b in cluster)
    })

df_clusters = pd.DataFrame(cluster_list)

# Сортируем
df_clusters = df_clusters.sort_values(
    by="Кол-во брендов",
    ascending=False
)

df_clusters.to_excel(OUTPUT_FILE, index=False)
print(f"Готово! Кластеры сохранены в {OUTPUT_FILE}")

Готово! Кластеры сохранены в brands_duplicates_clusters.xlsx


## Database cleaning

In [125]:
INPUT_FILE = "data (41).xlsx"
OUTPUT_FILE = "brands_cleaned.xlsx"

df = pd.read_excel(INPUT_FILE)
df = df.loc[df["Название сделки"].notna()]

# убираем префиксы "Автосделка: " и "Autolead: "
df["Название сделки"] = df["Название сделки"].str.replace(r"^(Автосделка: |Autolead: )", "", regex=True, case=False)

# удаляем все строки, начинающиеся с "Тест"
df = df.loc[~df["Название сделки"].str.contains(r"^тест", case=False, regex=True)]


# ===== Убираем пробелы в названиях колонок =====
df.columns = df.columns.str.strip()

# Делаем datetime
df["Дата закрытия"] = pd.to_datetime(df["Дата закрытия"], errors="coerce")

# Заменяем заглушку 9999-12-31 на пустое
df.loc[df["Дата закрытия"] == pd.Timestamp("9999-12-31"), "Дата закрытия"] = pd.NaT


# ===== Общие функции =====
def is_garbage(text):
    if pd.isna(text):
        return True
    t = str(text).lower().strip()
    garbage = ["?", "??", "-", ".", "/", "\\", "нет", "блок", "ут", "уточ", "уточняю", "http://нет", "http://нет ", "http://-", 
               "http://- ", "http://.", "http://,", "http://уточняю у бренда", "http://уточняю", "http://1", "http://http:///", 
               "http://http:/// ", "Жду инфу от бренда", "http://жду инфо от бренда", "жду", "http://жду", 
               "http://Жду инфу от бренда", "http://уточняю у бренда ", "http://=", "http://0", " "]
    
    return any(t == g or t.startswith(g + " ") for g in garbage)

def unique_join(values):
    result = []
    for v in values:
        if not v:
            continue
        parts = [p.strip() for p in str(v).split("|")]
        for p in parts:
            if p and p not in result:
                result.append(p)
    return " | ".join(result)

# ===== Instagram =====
def normalize_instagram_url(url):
    try:
        parsed = urlparse(url)
        path = parsed.path.strip("/")
        if not path:
            return ""
        username = path.split("/")[0]
        return f"https://instagram.com/{username}/"
    except:
        return ""

def clean_instagram(value):
    if is_garbage(value):
        return ""
    v = str(value).strip()
    v = re.sub(r"^блок\s*", "", v, flags=re.IGNORECASE)
    tokens = re.split(r"\s+", v)
    cleaned = []

    for token in tokens:
        token = token.strip()
        if not token:
            continue
        if token.startswith("#"):
            cleaned.append(token)
            continue
        if token.startswith("@"):
            token = token[1:]
        if "instagram.com" in token:
            norm = normalize_instagram_url(token)
            if norm and norm != "https://www.instagram.com/":  # фильтруем пустую
                cleaned.append(norm)
            continue
        if re.match(r"^[a-zA-Z0-9_.]+$", token):
            cleaned.append(f"https://instagram.com/{token}/")

    unique = []
    for x in cleaned:
        if x not in unique:
            unique.append(x)
    return " | ".join(unique)

# ===== Website =====
def normalize_url(url):
    try:
        parsed = urlparse(url)
        clean = urlunparse((parsed.scheme or "https",
                            parsed.netloc,
                            parsed.path.rstrip("/") + "/",
                            "", "", ""))
        return clean
    except:
        return ""

EMPTY_DOMAINS = {
    "www.instagram.com",
    "instagram.com",
    "www.ozon.ru",
    "ozon.ru"
}

def clean_site(value):
    if is_garbage(value):
        return ""
    v = str(value).strip()

    if not v.startswith("http"):
        v = "https://" + v

    norm = normalize_url(v)

    # фильтруем битые URL
    if norm in ["http://http/", "http:///"]:
        return ""

    parsed = urlparse(norm)
    if not parsed.netloc:
        return ""

    # фильтруем пустые сайты из списка
    if parsed.netloc.lower() in EMPTY_DOMAINS:
        return ""

    return norm

# ===== VK =====
def normalize_vk_url(url):
    try:
        parsed = urlparse(url)
        domain = parsed.netloc.replace("m.vk.com", "vk.com")
        path = parsed.path.strip("/")
        if not path:
            return ""
        return f"https://vk.com/{path}"
    except:
        return ""

def clean_vk(value):
    if is_garbage(value):
        return ""
    v = str(value).strip()
    v = re.sub(r"^блок\s*", "", v, flags=re.IGNORECASE)
    if v.startswith("#"):
        v = v[1:]
    if "vk.com" not in v:
        v = f"https://vk.com/{v.replace('@','')}"
    return normalize_vk_url(v)

# ===== Telegram =====
def clean_tg(value):
    if is_garbage(value):
        return ""

    v = str(value).strip().replace("@", "")

    # формируем ссылку
    if not v.startswith("http"):
        v = f"https://t.me/{v.split('/')[-1]}"

    if "t.me" not in v:
        username = v.split("/")[-1]
        v = f"https://t.me/{username}"

    # убираем пустые ссылки типа https://t.me/
    if v.rstrip("/") == "https://t.me":
        return ""

    return v

# ===== Применяем очистку =====
for col, func in [
    ("Instagram/хэштег", clean_instagram),
    ("Ссылка на сайт", clean_site),
    ("Вконтакте бренда", clean_vk),
    ("Телеграм бренда", clean_tg)
]:
    if col in df.columns:
        df[col] = df[col].apply(func)

# ===== Перекладываем ссылки =====
def route_links(row):
    ig = row.get("Instagram/хэштег", "")
    site = row.get("Ссылка на сайт", "")
    vk = row.get("Вконтакте бренда", "")
    tg = row.get("Телеграм бренда", "")

    def classify(link):
        if not link:
            return None
        if "vk.com" in link:
            return "vk"
        if "t.me" in link:
            return "tg"
        if "instagram.com" in link or link.startswith("#"):
            return "ig"
        if link.startswith("http"):
            return "site"
        return "ig"

    for value in [ig, site, vk, tg]:
        target = classify(value)
        if target == "ig":
            ig = unique_join([ig, value])
        elif target == "site":
            site = unique_join([site, value])
        elif target == "vk":
            vk = unique_join([vk, value])
        elif target == "tg":
            tg = unique_join([tg, value])
    return pd.Series([ig, site, vk, tg])

df[[
    "Instagram/хэштег",
    "Ссылка на сайт",
    "Вконтакте бренда",
    "Телеграм бренда"
]] = df.apply(route_links, axis=1)

# ===== Группировка по бренду =====

def merge_sites(values):
    urls = []

    for v in values:
        if not v:
            continue
        parts = [p.strip() for p in str(v).split("|")]

        for p in parts:
            norm = normalize_url(p)
            if norm:
                parsed = urlparse(norm)
                netloc = parsed.netloc
                # добавляем www., если его нет
                if not netloc.startswith("www."):
                    netloc = "www." + netloc
                root_url = f"{parsed.scheme}://{netloc}/"
                urls.append(root_url)

    # оставляем только уникальные корневые URL
    urls = list(dict.fromkeys(urls))
    return " | ".join(urls)


grouped = df.groupby("Бренд").agg({
    "Instagram/хэштег": unique_join,
    "Ссылка на сайт": merge_sites,
    "Вконтакте бренда": unique_join,
    "Телеграм бренда": unique_join
}).reset_index()

# ===== Последняя сделка по бренду =====
df["Дата создания"] = pd.to_datetime(df.get("Дата создания"), errors="coerce")

# колонки, которые хотим взять
required = [
    "Бренд", "Название сделки", "amoCRM link", "Дата создания", "Дата закрытия", "Имя", "Блок Instagram", "Сегмент",
    "Тип бренда", "Известность бренда", "Направленность",
    "Правильное произношение бренда дл"
]

# проверяем, есть ли колонки в df
available_cols = [c for c in required if c in df.columns]

idx = df.groupby("Бренд")["Дата создания"].idxmax()
latest_deals = df.loc[idx, available_cols]

# ===== Объединяем с grouped =====
final = grouped.merge(latest_deals, on="Бренд", how="left")

# ===== Сохраняем =====
final.to_excel(OUTPUT_FILE, index=False)
print("Готово! База очищена и последняя сделка добавлена 🎉")



C:\Users\varva\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Готово! База очищена и последняя сделка добавлена 🎉


## Web sites activity check

In [ ]:
session = requests.Session()


session.headers.update(HEADERS)


# ---------- проверка сайта ----------

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
}

def site_exists(url):

    def try_request(test_url):
        try:
            r = session.get(
                test_url,
                headers=HEADERS,
                timeout=12,
                allow_redirects=True,
                verify=False
            )

            # если сайт отвечает чем угодно кроме 5xx — считаем живым
            if r.status_code < 500:
                return True, r.url

        except:
            pass

        return False, test_url

    url = str(url).strip()

    if not url.startswith("http"):
        url = "https://" + url

    ok, final_url = try_request(url)

    if ok:
        return True, final_url

    http_url = url.replace("https://", "http://")
    return try_request(http_url)



# ---------- поиск новостей ----------

import requests
import re
from bs4 import BeautifulSoup
from datetime import datetime

NEWS_PATHS = [

    # --- базовые ---
    "/blog",
    "/news",
    "/articles",
    "/press",
    "/media",
    "/community",

    # --- русские аналоги ---
    "/blog/",
    "/news/",
    "/novosti",
    "/novosti/",
    "/newsroom",
    "/press-center",
    "/presscenter",

    # --- русские названия ---
    "/блог",
    "/новости",
    "/статьи",
    "/пресс",
    "/сми",

    # --- e-commerce ---
    "/journal",
    "/stories",
    "/updates",
    "/insights",
    "/magazine",

    # --- маркетинг / бренды ---
    "/about/news",
    "/company/news",
    "/company/blog",
    "/academy",
    "/education",
    "/events",

    # --- CMS стандартные ---
    "/index.php/blog",
    "/blog.html",
    "/news.html"
]

MONTHS_EN = {
    "january": 1, "jan": 1,
    "february": 2, "feb": 2,
    "march": 3, "mar": 3,
    "april": 4, "apr": 4,
    "may": 5,
    "june": 6, "jun": 6,
    "july": 7, "jul": 7,
    "august": 8, "aug": 8,
    "september": 9, "sep": 9,
    "october": 10, "oct": 10,
    "november": 11, "nov": 11,
    "december": 12, "dec": 12
}

MONTHS_RU = {
    "января": 1, "февраля": 2, "марта": 3, "апреля": 4,
    "мая": 5, "июня": 6, "июля": 7, "августа": 8,
    "сентября": 9, "октября": 10, "ноября": 11, "декабря": 12
}

DATE_PATTERNS = [
    r"\d{1,2}\.\d{1,2}\.\d{4}",  # 23.10.2025
    r"\d{4}-\d{2}-\d{2}",        # 2025-10-23
]

RU_PATTERN = re.compile(
    r"(\d{1,2})\s+(" + "|".join(MONTHS_RU.keys()) + r")\s+(\d{4})"
)

RU_NO_YEAR = re.compile(
    r"(\d{1,2})\s+(" + "|".join(MONTHS_RU.keys()) + r")"
)

EN_NO_YEAR = re.compile(
    r"(\d{1,2})\s+(" + "|".join(MONTHS_EN.keys()) + r")",
    re.IGNORECASE
)

from datetime import datetime

def guess_year(day, month):
    """ Определяем год, если он не указан"""

    today = datetime.today()

    candidate = datetime(today.year, month, day)

    # если дата в будущем → значит прошлый год
    if candidate > today:
        return today.year - 1

    return today.year


def parse_date(text):

    text = text.lower().strip()

    # dd.mm.yyyy
    try:
        return datetime.strptime(text, "%d.%m.%Y")
    except:
        pass

    # yyyy-mm-dd
    try:
        return datetime.strptime(text, "%Y-%m-%d")
    except:
        pass

    # русский формат С ГОДОМ
    match = RU_PATTERN.search(text)
    if match:
        d, m, y = match.groups()
        return datetime(int(y), MONTHS_RU[m], int(d))

    # русский БЕЗ года
    match = RU_NO_YEAR.search(text)
    if match:
        d, m = match.groups()
        year = guess_year(int(d), MONTHS_RU[m])
        return datetime(year, MONTHS_RU[m], int(d))

    # английский БЕЗ года
    match = EN_NO_YEAR.search(text)
    if match:
        d, m = match.groups()
        month = MONTHS_EN[m.lower()]
        year = guess_year(int(d), month)
        return datetime(year, month, int(d))

    return None


def extract_dates_from_page(url):
    try:
        r = session.get(url, timeout=8, verify=False)
        soup = BeautifulSoup(r.text, "html.parser")

        dates = []

        # ---------- ищем <time datetime="..."> ----------
        for t in soup.find_all("time"):
            dt_attr = t.get("datetime")
            if dt_attr:
                dt = parse_date(dt_attr)
                if dt:
                    dates.append(dt)
            else:
                text = t.get_text(strip=True)
                dt = parse_date(text)
                if dt:
                    dates.append(dt)

        # ---------- ищем <span class="date"> ----------
        for s in soup.find_all("span", class_="date"):
            text = s.get_text(strip=True)
            dt = parse_date(text)
            if dt:
                dates.append(dt)

        # ---------- fallback: весь текст страницы ----------
        if not dates:
            text = soup.body.get_text(" ") if soup.body else soup.get_text(" ")
            for pattern in DATE_PATTERNS + [RU_PATTERN, RU_NO_YEAR, EN_NO_YEAR]:
                for match in re.findall(pattern, text):
                    dt = parse_date(match)
                    if dt:
                        dates.append(dt)

        return max(dates) if dates else None

    except:
        return None


def get_latest_news_date(site):

    for path in NEWS_PATHS:
        url = site.rstrip("/") + path
        dt = extract_dates_from_page(url)

        if dt:
            return dt.strftime("%d.%m.%Y")

    return "нет новостей"


# ---------- обработка ячейки ----------

def check_sites_cell(cell):

    # ---------- пустая ячейка ----------
    if pd.isna(cell) or not str(cell).strip():
        return pd.Series({"status": "", "news": ""})

    parts = [x.strip() for x in str(cell).split("|") if x.strip()]

    exists = []
    not_exists = []
    news_results = []

    # ---------- обработка сайтов ----------
    for site in parts:

        ok, normalized = site_exists(site)

        # ❌ сайт НЕ существует → только статус
        if not ok:
            not_exists.append(site)
            continue  # ← ключевая строка: новости НЕ ищем

        # ✅ сайт существует
        exists.append(normalized)

        date = get_latest_news_date(normalized)

        if len(parts) == 1:
            news_results.append(date)
        else:
            news_results.append(f"{normalized}: {date}")

        time.sleep(random.uniform(1, 2))  # безопасная пауза

    # ---------- форматирование результата ----------

    # 👉 один сайт
    if len(parts) == 1:

        if exists:
            status = "да"
            news = news_results[0] if news_results else ""
        else:
            status = "нет"
            news = ""

    # 👉 несколько сайтов
    else:

        blocks = []

        if exists:
            blocks.append("Сайт существует: " + ", ".join(exists))

        if not_exists:
            blocks.append("Сайт не существует: " + ", ".join(not_exists))

        status = "\n".join(blocks)

        # если живых сайтов нет → новости пусто
        news = " | ".join(news_results) if news_results else ""

    return pd.Series({"status": status, "news": news})

In [ ]:
INPUT_FILE = "brands_cleaned.xlsx"
df = pd.read_excel(INPUT_FILE)

In [ ]:
MAX_WORKERS = 6  # можно 8–16

# ---------- обработка одного сайта ----------
def process_site(site):

    # ---------- 1. пустая ячейка ----------
    if pd.isna(site) or str(site).strip() == "":
        return "", ""

    # ---------- 2. разбиваем ссылки ----------
    urls = [u.strip() for u in str(site).split("|") if u.strip()]

    exists = []
    not_exists = []
    news_info = []

    # ---------- 3. проверяем каждый сайт ----------
    for url in urls:

        ok, normalized = site_exists(url)

        if ok:
            exists.append(normalized)

            date = get_latest_news_date(normalized)

            if len(urls) == 1:
                # формат для одного сайта
                news_info.append(date)
            else:
                news_info.append(f"{normalized}: {date}")

        else:
            not_exists.append(url)

    # ==================================================
    # ---------- 4. если сайт один ----------
    # ==================================================

    if len(urls) == 1:

        if exists:
            return "да", news_info[0]
        else:
            return "нет", ""

    # ==================================================
    # ---------- 5. если сайтов несколько ----------
    # ==================================================

    status_blocks = []

    if exists:
        status_blocks.append(
            "Сайт существует: " + ", ".join(exists)
        )

    if not_exists:
        status_blocks.append(
            "Сайт не существует: " + ", ".join(not_exists)
        )

    status = "; ".join(status_blocks)
    news = "; ".join(news_info) if news_info else ""

    return status, news

In [ ]:
sites = df_test["Ссылка на сайт"].tolist()
results = []

with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:

    for i, result in enumerate(
        tqdm(executor.map(process_site, sites), total=len(sites))
    ):
        results.append(result)

        # автосохранение каждые 50 строк
        if i % 50 == 0:
            tmp = pd.DataFrame(
                results,
                columns=["Сайт существует", "Дата последней новости"]
            )

            df_test.loc[:i, ["Сайт существует", "Дата последней новости"]] = tmp.values
            df_test.to_excel("autosave.xlsx", index=False)

# финальная запись
tmp = pd.DataFrame(results, columns=["Сайт существует", "Дата последней новости"])
df_test[["Сайт существует", "Дата последней новости"]] = tmp.values
df_test.to_excel("brands_sites_news.xlsx", index=False)

print("Готово!")